In [1]:
import torch
import pandas as pd
import plotly.express as px
import plotly.io as pio

from types import SimpleNamespace

from PChainger_Parser import PChainger_Parser
from Electricity_model import ELECTRICITY
from NILM_Dataset import NILMDataset

torch.set_default_tensor_type(torch.DoubleTensor)
pio.renderers.default = 'iframe' # or 'colab' or 'iframe' or 'iframe_connected' or 'sphinx_gallery'

In [2]:
data_file = "./data/test/geert/main.csv"
model_file = "./results_UK-DALE_TitanV_kettle/uk_dale/kettle/best_acc_model.pth"

device = "cpu"

In [3]:
args = SimpleNamespace(
    drop_out=0.1,
    heads=2,
    hidden=256,
    n_layers=2,
    output_size=1,
    pretrain=False,
    window_size=480,
)

In [5]:
# `drop_last = True` will drop the last part of the data that
# does not fit in the window_size anymore!

ds_parser = PChainger_Parser(
    sampling="6s",
    normalize="mean",
    cutoff=[3100],
    drop_last=True,
    #threshold=[2000],
    location=data_file,
    #min_on=[2],
    #min_off=[0],
    val_size=1,
    separator=",",
    window_size=480,
    window_stride=240
)

In [6]:
model = ELECTRICITY(args)
model.load_state_dict(torch.load(model_file, map_location=torch.device(device)))

<All keys matched successfully>

In [7]:
inference_data = NILMDataset(
    x=ds_parser.x,
    y=[0] * len(ds_parser.x),
    status=[0] * len(ds_parser.x),
    window_size=480,
    stride=480
)

In [8]:
inference_dataloader = torch.utils.data.DataLoader(
    dataset=inference_data,
    batch_size=480,
)

In [9]:
out = model.eval()

In [10]:
batches = []
for batch in inference_dataloader:
    batches.append(model(batch[0].to("cpu")))

data = []
for batch in batches:
    for chunk in batch[0]:
        data += list(chunk[0].tolist())

In [11]:
df = pd.DataFrame(data, index=ds_parser.index, columns=["inference"])
df

,inference
time,
2023-04-23 10:03:24,0.010961
2023-04-23 10:03:30,0.005562
2023-04-23 10:03:36,0.009228
2023-04-23 10:03:42,0.005534
2023-04-23 10:03:48,0.009228
...,...
2023-05-02 13:14:54,0.005263
2023-05-02 13:15:00,0.009099
2023-05-02 13:15:06,0.005245


In [12]:
df_inf = pd.DataFrame(dict(
    index = ds_parser.index,
    inferred = data
))

fig = px.line(df_inf, x="index", y="inferred", line_shape="linear", render_mode="svg")
fig.show()

In [13]:
kettle = pd.read_csv("./data/test/geert/kettle.csv", sep=',', header=None, names=["time", "aggregate"])
kettle["time"] = pd.to_datetime(kettle["time"])
kettle = kettle.set_index("time")

In [14]:
kettle[ds_parser.index[0]:ds_parser.index[-1]].index

DatetimeIndex(['2023-04-23 10:03:26', '2023-04-23 10:03:27',
               '2023-04-23 10:03:28', '2023-04-23 10:03:29',
               '2023-04-23 10:03:30', '2023-04-23 10:03:31',
               '2023-04-23 10:03:33', '2023-04-23 10:03:34',
               '2023-04-23 10:03:35', '2023-04-23 10:03:36',
               ...
               '2023-05-02 13:15:07', '2023-05-02 13:15:08',
               '2023-05-02 13:15:09', '2023-05-02 13:15:11',
               '2023-05-02 13:15:12', '2023-05-02 13:15:13',
               '2023-05-02 13:15:14', '2023-05-02 13:15:15',
               '2023-05-02 13:15:17', '2023-05-02 13:15:18'],
              dtype='datetime64[ns]', name='time', length=354717, freq=None)

In [15]:
df_kettle = pd.DataFrame(dict(
    index = kettle[ds_parser.index[0]:ds_parser.index[-1]].index,
    data = kettle[ds_parser.index[0]:ds_parser.index[-1]]["aggregate"],
))

fig = px.line(df_kettle, x="index", y="data", line_shape="linear", render_mode="svg")
fig.show()

In [16]:
main = pd.read_csv(data_file, sep=',', header=None, names=["time", "aggregate"])
main["time"] = pd.to_datetime(main["time"])
main = main.set_index("time")

In [17]:
df_main = pd.DataFrame(dict(
    index = main[ds_parser.index[0]:ds_parser.index[-1]].index,
    data = main[ds_parser.index[0]:ds_parser.index[-1]]["aggregate"],
))

fig = px.line(df_main, x="index", y="data", line_shape="linear", render_mode="svg")
fig.show()